In [ ]:
import colormaps as cmaps
import json
import os
import re
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pycountry
import seaborn as sns
from IPython import get_ipython
from IPython.core.display import HTML
from IPython.core.interactiveshell import InteractiveShell
from publib import set_style, fix_style
sns.set_style("whitegrid")
set_style(['origin'])
InteractiveShell.ast_node_interactivity = "all"

import stata_setup
stata_setup.config('/home/jovyan/kay/stata_bin', 'mp', splash=False)
from pystata import stata

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 20)
pd.set_option('display.max_rows', 120)
pd.set_option('display.expand_frame_repr', False)


df_soep = pd.read_parquet("df_soep2.gzip")
df_ivs = pd.read_parquet("ivs_unmerged.gzip")


In [ ]:
# Create variable to identify source dataset after merging
df_ivs["soep"] = 0
df_soep["soep"] = 1

# Adjust variable names for consistence and merging
df_ivs.rename(columns={"educ_casmin": "gen_casmin", "subj_health": "bad_health", "satisf_peers": "satisfaction"}, inplace=True)

# Transform variables
df_ivs['gen_regunempl'] = np.where(df_ivs['empl_status'] == 7, 1,
                                   np.where(df_ivs['empl_status'] < 0, np.nan, 0))

df_ivs['bad_health'] = df_ivs['bad_health'].apply(lambda x: 1 if x in [4, 5] else (0 if x in [1, 2, 3] else float('nan')))

# Assign IVS individuals unique ids for identification
df_ivs["pid"] = 0

start_int = int(df_soep['pid'].max()) + 1

num_missing = (df_ivs['pid'] == 0).sum()

new_pids = range(start_int, start_int + num_missing)

df_ivs.loc[df_ivs['pid'] == 0, 'pid'] = list(new_pids)

df = pd.concat([df_soep, df_ivs], axis=0)

# Drop missing data, use only countries with 50+ pids and max 10000
df = df.dropna(subset=["age", "sex", "gen_casmin", "bad_health", "soep", "gen_regunempl"])

valid_countries_df = df.groupby(["corigin_iso", "soep"])["pid"].nunique().reset_index()

valid_countries_df = valid_countries_df[valid_countries_df['pid'] >= 50]

valid_countries = valid_countries_df.groupby("corigin_iso").filter(lambda x: len(x) == 2)["corigin_iso"].unique()

df_valid_countries = df[df["corigin_iso"].isin(valid_countries)]


def sample_pids(x, n=10000):
    unique_pids = x["pid"].unique()
    sampled_pids = pd.Series(unique_pids).sample(min(len(unique_pids), n), random_state=1).tolist()
    return x[x["pid"].isin(sampled_pids)]

df_sampled = df_valid_countries.groupby("corigin_iso", group_keys=False).apply(sample_pids)


In [ ]:
# Load dataset into dataframe
cem = pd.read_stata("cem_weighted.dta")

In [ ]:
# Determine the columns in cem that are not in df
new_cols = [col for col in cem.columns if col not in df.columns]

# Merging datasets, only the new columns to prevent duplicates
merged_df = df.merge(cem[['pid', 'syear'] + new_cols], on=['pid', 'syear'], how='left')
merged_df.corigin_iso.astype('string');

In [ ]:
# Filter rows where cem_matched is 1
matched_df = merged_df[merged_df['cem_matched'] == 1]

# Calculate average values for each covariate for control (soep = 0)
control_avg = matched_df[matched_df['soep'] == 0].groupby('corigin_iso').mean().reset_index()

# Calculate average values for each covariate for treatment (soep = 1)
treatment_avg = matched_df[matched_df['soep'] == 1].groupby('corigin_iso').mean().reset_index()


In [ ]:
# Define the columns to check for imbalance
columns_to_check = ['age', 'bad_health', 'gen_casmin', 'syear', 'gen_regunempl']


# Function to calculate weighted mean and standard deviation
def weighted_stats(df, weight_col, columns):
    weighted_means = {}
    weighted_stds = {}
    for column in columns:
        weights = df[weight_col]
        values = df[column]
        mean = np.average(values, weights=weights)
        variance = np.average((values-mean)**2, weights=weights)
        std = np.sqrt(variance)
        
        weighted_means[column] = mean
        weighted_stds[column] = std

    return weighted_means, weighted_stds

# Initializing an empty list to store SMD values rows
smd_rows = []
matched_df_grouped = matched_df.groupby('corigin_iso')

for name, group in matched_df_grouped:
    control_group = group[group['soep'] == 0]
    treatment_group = group[group['soep'] == 1]
    
    control_means, control_stds = weighted_stats(control_group, 'cem_weights', columns_to_check)
    treatment_means, treatment_stds = weighted_stats(treatment_group, 'cem_weights', columns_to_check)
    
    # Calculating the SMD
    epsilon = 1e-8  # Prevent division by zero
    smds = {}

    for column in columns_to_check:
        mean_diff = abs(treatment_means[column] - control_means[column])  # Taking absolute value
        pooled_sd = np.sqrt((control_stds[column]**2 + treatment_stds[column]**2)/2 + epsilon)
        smds[f'smd_{column}'] = abs(mean_diff / pooled_sd)  # Taking absolute value

    smd_rows.append({'corigin_iso': name, **smds})

smd_df = pd.DataFrame(smd_rows)

smd_df['average_smd'] = smd_df.iloc[:, 1:].mean(axis=1)

In [ ]:
# Check balance and remove countries with a variable having an imbalance > 0.2
def create_love_plot(smd_df, columns_to_check, country):
    country_smd = smd_df[smd_df['corigin_iso'] == country]
   
    smd_columns = [f'smd_{col}' for col in columns_to_check]
    
    #plt.figure(figsize=(10, 5))
    #plt.barh(columns_to_check, country_smd[smd_columns].values.flatten())
    
    #plt.xlabel('Standardized Mean Differences (SMDs)')
    #plt.title(f'Love Plot for {country}')
    #plt.axvline(x=0.1, color='red', linestyle='--', label='Threshold (0.1)')
    #plt.axvline(x=-0.1, color='red', linestyle='--')
    #plt.legend()
    #plt.gca().invert_yaxis()  # Invert y-axis to have the first covariate at the top
    #plt.show()
    
    if any(abs(country_smd[smd_columns].values.flatten()) > 0.2):
        return country
    return None

countries_to_remove = []
countries = smd_df['corigin_iso'].unique()
for country in countries:
    country_with_imbalance = create_love_plot(smd_df, columns_to_check, country)
    if country_with_imbalance:
        countries_to_remove.append(country_with_imbalance)

smd_df_filtered = smd_df[~smd_df['corigin_iso'].isin(countries_to_remove)]


In [ ]:
from scipy.stats import ttest_ind

# Use t-test to determine, if difference in life satisfaction is significant
results = []

def weighted_mean(group, value_col, weight_col):
    return (group[value_col] * group[weight_col]).sum() / group[weight_col].sum()

avg_satisfaction = matched_df.groupby(['corigin_iso', 'soep']).apply(weighted_mean, 'satisfaction', 'cem_weights').reset_index(name='weighted_satisfaction')

results_dict = {}

for country in avg_satisfaction['corigin_iso'].unique():
    country_data = matched_df[matched_df['corigin_iso'] == country]

    control_group = country_data[country_data['soep'] == 0]['satisfaction']
    treatment_group = country_data[country_data['soep'] == 1]['satisfaction']

    t_stat, p_value = ttest_ind(control_group, treatment_group, equal_var=False)

    results_dict[country] = {
        'T-statistic': t_stat,
        'P-value': round(p_value, 3)
    }

significant_countries = [k for k, v in results_dict.items() if v['P-value'] < 0.1]

In [ ]:
# Assign asterisks to plot according to significance level

significance_dict = {}
for k, v in results_dict.items():
    if v['P-value'] < 0.01:
        significance_dict[k] = '***'
    elif v['P-value'] < 0.05:
        significance_dict[k] = '**'
    elif v['P-value'] < 0.1:
        significance_dict[k] = '*'
    else:
        significance_dict[k] = ''

matched_df = merged_df[merged_df['cem_matched'] == 1]

def weighted_mean(group, value_col, weight_col):
    return (group[value_col] * group[weight_col]).sum() / group[weight_col].sum()

avg_satisfaction = matched_df.groupby(['corigin_iso', 'soep']).apply(weighted_mean, 'satisfaction', 'cem_weights').reset_index(name='weighted_satisfaction')

# Remove Germans
plot_data = avg_satisfaction[avg_satisfaction['corigin_iso'] != 'DEU']

plt.figure(figsize=(10, 14))

num_countries = len(plot_data['corigin_iso'].unique())
country_colors = cmaps.bold(np.linspace(0, 1, num_countries))
country_palette = dict(zip(plot_data['corigin_iso'].unique(), country_colors))

# Create lines connecting the dots
for index, country in enumerate(plot_data['corigin_iso'].unique()):
    country_data = plot_data[plot_data['corigin_iso'] == country]
    

    if len(country_data) == 2:
        plt.plot(country_data['weighted_satisfaction'], country_data['corigin_iso'], color=country_palette[country], linestyle='-', zorder=1, linewidth=1.0)

# Create scatter plots using different markers for Control and Treatment
sns.scatterplot(data=plot_data, y='corigin_iso', x='weighted_satisfaction', hue='corigin_iso', palette=country_palette, style='soep', markers=['.', 's'], s=100, zorder=2, legend=False)

countries = plot_data['corigin_iso'].unique()
modified_countries = [country + significance_dict.get(country, '') for country in countries]

plt.yticks(ticks=np.arange(len(countries)), labels=modified_countries, rotation=0, fontsize=10)  # Use modified country labels

plt.xticks()  
plt.ylabel('', fontsize=12) 
plt.xlabel('') 
plt.ylim(len(countries) - 0.5, -0.5)
plt.grid(color='k', linestyle='--', linewidth=0.8) 
plt.tight_layout()

plt.savefig("matching.pdf")
plt.show();

In [ ]:
control_satisfaction = plot_data[plot_data['soep'] == 0].set_index('corigin_iso')['weighted_satisfaction']
treatment_satisfaction = plot_data[plot_data['soep'] == 1].set_index('corigin_iso')['weighted_satisfaction']
difference_satisfaction = treatment_satisfaction - control_satisfaction

# Combine the differences with the significance levels
difference_df = pd.DataFrame(difference_satisfaction).rename(columns={'weighted_satisfaction': 'difference'})
difference_df['significance'] = difference_df.index.map(significance_dict)

# Sort by significance first and then by the absolute difference
difference_df['abs_difference'] = difference_df['difference'].abs()
difference_df.sort_values(by=['significance', 'abs_difference'], ascending=[False, False], inplace=True)
difference_df.drop('abs_difference', axis=1, inplace=True)

print(difference_df)

In [ ]:
# Display percentage of missing values for a selection of variables

variables = ['satisfaction', 'gen_wpartner', 'gen_seppart', 'gen_wid_div', 'bad_health', 
             'continent_Asia', 'continent_North_America', 'continent_South_America', 'continent_Oceania', 'discrimination', 
             'social', 'feel_german', 'gen_vocation', 'gen_retired', 'gen_regunempl',
             'rel_christ', 'rel_musl', 'edu_years', 'refugee', 'german', 'lang_profic', 
             "xenophobia", "syear", "soep", "cohort_18_29", "cohort_45_59", "cohort_60", "gen_income", "sex", "corigin_iso"]


# Filter the data for soep = 1

data_soep_1 = matched_df[(matched_df['soep'] == 1)][variables]

# Calculate missing data percentage

missing_data_percent = data_soep_1.groupby('syear').apply(lambda x: x.isnull().mean() * 100).drop(columns='syear')

plt.figure(figsize=(20, 12))

sns.heatmap(missing_data_percent, cmap='viridis_r', annot=True, fmt='.2f', 

            cbar_kws={'label': 'Missing Data Percentage'}, annot_kws={"size": 8})

plt.title('Percentage of Missing Data by Year and Variable (SOEP = 1, corigin_iso != DEU)', fontsize=14)

plt.xlabel('Variable', fontsize=12)

plt.ylabel('Year', fontsize=12)

plt.xticks(rotation=45, ha='right', fontsize=10)

plt.yticks(fontsize=10)

plt.tight_layout()

plt.show();

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Save the original index
original_index = data_soep_1.index

# Store the 'corigin_iso' column
corigin_iso_column = matched_df.loc[original_index, 'corigin_iso'].copy()

# Exclude corigin_iso from data_soep_1
data_soep_1 = data_soep_1.drop(columns=['corigin_iso'])

# Impute missing values in data_soep_1
imputer = IterativeImputer(random_state=0)
imputed_array = imputer.fit_transform(data_soep_1)

# Convert the output into DataFrame and assign column names
data_soep_1_imputed = pd.DataFrame(imputed_array, columns=data_soep_1.columns, index=original_index)

# Update original matched_df with the imputed values
for col in data_soep_1.columns:
    matched_df.loc[original_index, col] = data_soep_1_imputed.loc[:, col]

# add the corigin_iso to the imputed DataFrame
matched_df.loc[original_index, 'corigin_iso'] = corigin_iso_column


In [ ]:
# Display percentage of missing values for a selection of variables

variables = ['satisfaction', 'gen_wpartner', 'gen_seppart', 'gen_wid_div', 'bad_health', 
             'continent_Asia', 'continent_North_America', 'continent_South_America', 'continent_Oceania', 'discrimination', 
             'social', 'feel_german', 'gen_vocation', 'gen_retired', 'gen_regunempl',
             'rel_christ', 'rel_musl', 'edu_years', 'refugee', 'german', 'lang_profic', 
             "xenophobia", "syear", "soep", "cohort_18_29", "cohort_45_59", "cohort_60", "gen_income", "refugee", "sex"]


# Filter the data for soep = 1

data_soep_1 = matched_df[(matched_df['soep'] == 1)][variables]

# Calculate missing data percentage

missing_data_percent = data_soep_1.groupby('syear').apply(lambda x: x.isnull().mean() * 100).drop(columns='syear')



plt.figure(figsize=(20, 12))

sns.heatmap(missing_data_percent, cmap='viridis_r', annot=True, fmt='.2f', 

            cbar_kws={'label': 'Missing Data Percentage'}, annot_kws={"size": 8})

plt.title('Percentage of Missing Data by Year and Variable (SOEP = 1, corigin_iso != DEU)', fontsize=14)

plt.xlabel('Variable', fontsize=12)

plt.ylabel('Year', fontsize=12)

plt.xticks(rotation=45, ha='right', fontsize=10)

plt.yticks(fontsize=10)

plt.tight_layout()

plt.show();

In [ ]:
%%stata -d matched_df -force

destring sex, replace

* Save a copy of the original dataset
save "original_dataset.dta", replace

* Run the regression model
regress satisfaction sex gen_wpartner gen_seppart gen_wid_div bad_health discrimination social feel_german gen_vocation gen_retired gen_regunempl rel_christ rel_musl edu_years refugee german syear cohort_18_29 cohort_45_59 cohort_60 gen_income, robust

* Save residuals in a new column
predict satisfaction_residual, residuals

* Save this modified dataset with residuals
save "modified_dataset.dta", replace


In [ ]:
# Reimport dataframe from Stata
merged_df = pd.read_stata("modified_dataset.dta")

In [ ]:
# Import WIF
freedom = pd.read_csv("freedom.csv").rename(columns={'iso_alpha3': 'corigin_iso', 'year': 'syear'})

In [ ]:
# Import Hofstede
six_dim = pd.read_csv("6dimensions.csv", sep=";").rename(columns={"ctr": "corigin_iso"}).drop("country", axis=1)

In [ ]:
# Some of this code is not redundant and not used for the thesis

from adjustText import adjust_text

# Function to assign asterisks based on p-value
def assign_asterisks(p_value):
    if p_value < 0.01: return '***'
    elif p_value < 0.05: return '**'
    elif p_value < 0.1: return '*'
    else: return ''

# Function to calculate weighted mean
def weighted_mean(group, value_col, weight_col):
    return (group[value_col] * group[weight_col]).sum() / group[weight_col].sum()

# Calculate the weighted means and differences
country_means = merged_df.groupby(['corigin_iso', 'soep']).apply(
    lambda group: pd.Series({
        'weighted_res_satisfaction': weighted_mean(group, 'satisfaction_residual', 'cem_weights'),
        'satisfaction': weighted_mean(group, 'satisfaction', 'cem_weights')
    })
).reset_index()


country_diffs = country_means.pivot(index='corigin_iso', columns='soep', values='satisfaction').diff(axis=1)[1]

# Merge the DataFrames to have one row per country
country_data = country_means[country_means['soep'] == 1].set_index('corigin_iso')
country_data['satisfaction_diff'] = country_diffs

# Assign the German satisfaction
deu_satisfaction = country_data.loc['DEU', 'weighted_res_satisfaction']

# Assign quadrants
conditions = [
    (country_data['weighted_res_satisfaction'] > deu_satisfaction) & (country_data['satisfaction_diff'] > 0),
    (country_data['weighted_res_satisfaction'] <= deu_satisfaction) & (country_data['satisfaction_diff'] > 0),
    (country_data['weighted_res_satisfaction'] <= deu_satisfaction) & (country_data['satisfaction_diff'] <= 0),
    (country_data['weighted_res_satisfaction'] > deu_satisfaction) & (country_data['satisfaction_diff'] <= 0)
]

values = ['Q1', 'Q2', 'Q3', 'Q4']
country_data['quadrant'] = np.select(conditions, values)

# Define color map
colors = list(cmaps.bold.colors) 

# Remove DEU from the plot
country_data_plot = country_data[country_data.index != 'DEU']

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = sns.scatterplot(data=country_data_plot.reset_index(), x='weighted_res_satisfaction', y='satisfaction_diff', hue='quadrant', palette=colors, s=100, hue_order=['Q1', 'Q2', 'Q3', 'Q4'])

leg = scatter.legend_
leg.set_title(None)
for text in leg.get_texts():
    text.set_fontsize('12')  # Set the fontsize here

texts = [plt.text(row['weighted_res_satisfaction'], row['satisfaction_diff'], row['corigin_iso'] + assign_asterisks(results_dict.get(row['corigin_iso'], {}).get('P-value', 1)), fontsize=8) for _, row in country_data_plot.reset_index().iterrows()]
adjust_text(texts)

plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.axvline(deu_satisfaction, color='gray', linestyle='--', linewidth=0.8)

plt.xlabel('Residualized Satisfaction (Reference Line: Natives)', fontsize=12)
plt.ylabel('Satisfaction Difference to Peers', fontsize=12)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.savefig("scatter_plot.pdf")
plt.show();



# Merge FIW and HDR data into working dataset
country_data_merged = merged_df.merge(freedom, on=['corigin_iso', 'syear'], how='left')
country_data_merged = country_data_merged.merge(six_dim, on=['corigin_iso'], how='left')

# Merging the quadrant column into country_data_merged
country_data_merged = country_data_merged.merge(
    country_data[['quadrant', 'satisfaction_diff']], 
    left_on='corigin_iso', 
    right_index=True,
    how='left'
)

# Define the indicators
indicators = ['PR_FIW', 'CL_FIW', "pdi", "idv", "mas", "uai", "ltowvs", "ivr"]

# Function to calculate the mean for each indicator and quadrant
def calculate_indicator_means(df):
    means = {}
    for indicator in indicators:
        # Attempt to convert values to numeric, non-numeric become NaN
        numeric_values = pd.to_numeric(df[indicator], errors='coerce')
        means[indicator] = numeric_values.mean()  # NaN values are excluded from the mean calculation
    return pd.Series(means)


# Applying the function to each quadrant
indicator_means_by_quadrant = country_data_merged.groupby('quadrant').apply(calculate_indicator_means).reset_index()

# Transposing the DataFrame
indicator_means_by_quadrant_T = indicator_means_by_quadrant.transpose()

# Renaming the columns
labels = {
    'quadrant': 'Quadrant',
    'PR_FIW': 'Political Rights',
    'CL_FIW': 'Civil Liberties',
    'pdi': 'Power Distance',
    'idv': 'Individualism',
    'mas': 'Masculinity',
    'uai': 'Uncertainty Avoidance',
    'ltowvs': 'Long Term Orientation',
    'ivr': 'Indulgence'
}

indicator_means_by_quadrant_T = indicator_means_by_quadrant_T.rename(labels)

# Display the transposed and renamed DataFrame
indicator_means_by_quadrant_T


# Labels for satisfaction variables

satisfaction_variable_labels = {

    'satisfaction': 'Residualized Satisfaction',

    'satisfaction_diff': 'Difference to Peers',

}


# Define the satisfaction variables and labels
satisfaction_variables = ['satisfaction', 'satisfaction_diff']

# Convert indicator columns to numeric, coercing errors to NaN
country_data_merged['PR_FIW'] = pd.to_numeric(country_data_merged['PR_FIW'], errors='coerce')
country_data_merged['CL_FIW'] = pd.to_numeric(country_data_merged['CL_FIW'], errors='coerce')
country_data_merged['pdi'] = pd.to_numeric(country_data_merged['pdi'], errors='coerce')
country_data_merged['idv'] = pd.to_numeric(country_data_merged['idv'], errors='coerce')
country_data_merged['mas'] = pd.to_numeric(country_data_merged['mas'], errors='coerce')
country_data_merged['uai'] = pd.to_numeric(country_data_merged['uai'], errors='coerce')
country_data_merged['ltowvs'] = pd.to_numeric(country_data_merged['ltowvs'], errors='coerce')
country_data_merged['ivr'] = pd.to_numeric(country_data_merged['ivr'], errors='coerce')

country_data_merged['PR_FIW'] = 7 - country_data_merged['PR_FIW']
country_data_merged['CL_FIW'] = 7- country_data_merged['CL_FIW']

# Update indicators
indicators_df = country_data_merged[indicators]

# Define the ranges for years in Germany
year_ranges = [(0,100)]

# Function to create and display a heatmap
def create_heatmap(data, year_range, title_suffix):
    filtered_data = data[data['y_in_germany'].between(*year_range)]
    correlation_matrix = filtered_data.corr(method='kendall').loc[satisfaction_variables, indicators].T

    plt.figure(figsize=(11, 8));
    heatmap = sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap=cmaps.cet_d_tritanopic_cwr, cbar=False, annot_kws={"size": 12})  

    heatmap.set_xticklabels([satisfaction_variable_labels.get(x, x) for x in correlation_matrix.columns], rotation=0, fontsize=12)  
    heatmap.set_yticklabels([labels.get(y, y) for y in correlation_matrix.index], rotation=0, fontsize=12)

    heatmap.xaxis.tick_top()  
    heatmap.xaxis.set_label_position('top') 

    title = ""
    heatmap.set_title(title, fontsize=11)

    plt.tight_layout()
    plt.savefig("correlation.pdf")
    plt.show();

# Loop over each year range
for year_range in year_ranges:
    create_heatmap(country_data_merged, year_range, '')
